In [31]:
import random
import heapq
import math

In [32]:
# Clase para generar una fuente discreta sin memoria basada en símbolos y probabilidades dados
class FuenteDiscretaSinMemoria:
    def __init__(self, simbolos_probabilidades):
        self.simbolos = list(simbolos_probabilidades.keys())  # Lista de símbolos
        self.probabilidades = list(simbolos_probabilidades.values())  # Probabilidades asociadas a los símbolos
        
    def generar_cadena(self, longitud):
        # Genera una cadena de símbolos de la longitud especificada, según las probabilidades dadas
        cadena = ''.join(random.choices(self.simbolos, weights=self.probabilidades, k=longitud))
        return cadena 

In [33]:
# Función para analizar las frecuencias de los símbolos en un texto
def analizaFrecuencias(texto):
    d = {}
    for a in texto:
        if a in d:
            d[a] += 1  # Incrementa la frecuencia del símbolo si ya existe en el diccionario
        else:
            d[a] = 1  # Si no existe, inicializa su frecuencia en 1
    return d

In [34]:
# Función auxiliar que toma los dos nodos con las frecuencias más bajas y los combina en un nuevo nodo
def minimosANodo(lista):
    min1 = heapq.heappop(lista)  # Extrae el nodo con la frecuencia mínima
    min2 = heapq.heappop(lista)  # Extrae el segundo nodo con la frecuencia mínima
    heapq.heappush(lista, (min1[0] + min2[0], min1[1], (min1[2], min2[2])))  # Inserta el nuevo nodo combinado en la lista

In [35]:
# Función para generar el árbol de Huffman dado un diccionario de frecuencias
def Hacer_arbol_huff(tabla):
    lista = [(p, k, (s,)) for k, (s, p) in enumerate(tabla.items())]  # Convierte la tabla de frecuencias a una lista de tuplas
    heapq.heapify(lista)  # Convierte la lista a una heap (cola de prioridad)
    while len(lista) > 1:
        minimosANodo(lista)  # Combina los dos nodos de menor frecuencia hasta que solo quede uno
    return lista[0][2]  # Devuelve el árbol generado

In [36]:
# Función para generar el código binario de Huffman a partir del árbol
def Hacer_codigo(arbol, prefijo=""):
    match len(arbol):
        case 1:
            return {arbol[0]: prefijo}  # Caso base: si el nodo es una hoja, asigna el prefijo actual como código
        case 2:
            # Caso recursivo: genera los códigos de los subárboles izquierdo y derecho con prefijos "0" y "1" respectivamente
            return Hacer_codigo(arbol[0], prefijo + "0") | Hacer_codigo(arbol[1], prefijo + "1")

In [37]:
# Función para guardar los códigos binarios generados en un archivo de texto
def guardar_codigos_binarios(codigo, archivo_salida='codigos_binarios.txt'):
    with open(archivo_salida, 'w', encoding='utf-8') as f:
        for simbolo, codigo_binario in codigo.items():
            f.write(f"{simbolo}: {codigo_binario}\n")  # Escribe cada símbolo y su código binario correspondiente
    print(f"Códigos binarios guardados en '{archivo_salida}'.")

In [38]:
# Función para guardar el árbol de Huffman de manera visual en un archivo de texto, mostrando los códigos binarios

def guardar_arbol_huffman(nodo, codigo=None, archivo_salida='arbol_huffman.txt', prefijo='', nivel=0):
    """
    Guarda el árbol de Huffman en un formato visual en un archivo de texto, incluyendo los códigos binarios.
    
    :param nodo: Nodo actual del árbol (puede ser un símbolo o una tupla de nodos).
    :param codigo: Diccionario de códigos binarios para cada símbolo.
    :param archivo_salida: Nombre del archivo de salida.
    :param prefijo: Prefijo actual ('0' o '1') para la visualización.
    :param nivel: Nivel de profundidad actual del árbol.
    """
    with open(archivo_salida, 'a', encoding='utf-8') as f:
        if isinstance(nodo, tuple) and len(nodo) == 2:  # Nodo intermedio con dos subnodos, isinstance() es una función en Python que se utiliza para comprobar si un objeto pertenece a una clase o tipo de dato en particular.
            # Nodo intermedio, sigue recursivamente
            guardar_arbol_huffman(nodo[0], codigo, archivo_salida, prefijo + '0', nivel + 1)
            f.write(f"{'    ' * nivel}Nodo intermedio\n")  # Valor del nodo intermedio
            guardar_arbol_huffman(nodo[1], codigo, archivo_salida, prefijo + '1', nivel + 1)
        elif isinstance(nodo, tuple) and len(nodo) == 1:  # Nodo hoja
            simbolo = nodo[0]
            codigo_binario = codigo[simbolo] if codigo and simbolo in codigo else ''
            f.write(f"{'    ' * nivel}'{simbolo}' ({codigo_binario})\n")
        else:  # Nodo único (es posible que no ocurra, pero se maneja por seguridad)
            f.write(f"{'    ' * nivel}'{nodo}'\n")

In [39]:
# Clase para la codificación y decodificación usando el algoritmo de Huffman -- Controlar

class Huffman:
    def __init__(self, frecs: dict):
        self.arbol = Hacer_arbol_huff(frecs)  # Genera el árbol de Huffman a partir de las frecuencias
        self.codigo = Hacer_codigo(self.arbol)  # Genera los códigos de Huffman a partir del árbol
        self.decodificacion = {v: k for k, v in self.codigo.items()}  # Crea diccionario inverso para decodificación

    def codifica(self, texto: str) -> str:
        # Codifica un texto usando los códigos de Huffman generados
        return ''.join(self.codigo[char] for char in texto)

    def decodifica(self, bits: str) -> str:
        # Decodifica una cadena de bits usando los códigos de Huffman generados
        resultado = []
        buffer = ""             # acumula bits mientras se decodifica la cadena codificada, cuando hay una coincidencia, lee el simbolo y reinicia el buffer
        for bit in bits:
            buffer += bit
            if buffer in self.decodificacion:
                resultado.append(self.decodificacion[buffer])       # se utiliza para añadir un elemento (símbolo decodificado) al final de una lista
                buffer = ""
        return ''.join(resultado)

In [40]:
def calcular_entropia(probabilidades):                      # Ver libro para formulas
    return -sum(p * math.log2(p) for p in probabilidades if p > 0)

In [41]:
# Función para evaluar el rendimiento del codificador Huffman -- VER ECUACIONES/RELACIONES EN LIBRO

def evaluar_rendimiento(cadena_original, cadena_codificada, simbolos_probabilidades, huffman):
    # Entropía de la fuente
    entropia_fuente = calcular_entropia(list(simbolos_probabilidades.values()))
    
    # Tamaño de la cadena original (suponiendo 8 bits por carácter en UTF-8)

    tamano_original = len(cadena_original) * 8      # probar
    
    # Tamaño de la cadena codificada (en bits)
    tamano_codificado = len(cadena_codificada)
    
    # Tasa de compresión    -- Verificar

    tasa_compresion = tamano_original / tamano_codificado
    
    print(f"Entropía de la fuente: {entropia_fuente:.4f} bits/símbolo")
    print(f"Tamaño original: {tamano_original} bits")
    print(f"Tamaño codificado: {tamano_codificado} bits")
    print(f"Tasa de compresión: {tasa_compresion:.4f}")
    
    # Verificar si cumple con el Teorema de Codificación de Fuente, Ver ecuacion

    longitud_media = sum(len(huffman.codigo[s]) * simbolos_probabilidades[s] for s in simbolos_probabilidades)
    print(f"Longitud media del código de Huffman: {longitud_media:.4f} bits/símbolo")
    
    if longitud_media >= entropia_fuente:   #Apuntes
        print("Cumple con el Teorema de Codificación de Fuente.")
    else:
        print("No cumple con el Teorema de Codificación de Fuente.")

In [42]:
# Función principal que ejecuta el flujo del objetivo 2.4
def ejecutar_objetivo_2_4():
    # Leer el archivo y analizar las frecuencias de los símbolos
    nombre_archivo = 'La_Divina_Comedia.txt'
    with open(nombre_archivo, 'r', encoding='utf-8') as f:
        texto = f.read()
    
    tabla_frecuencias = analizaFrecuencias(texto)
    total_simbolos = sum(tabla_frecuencias.values())
    simbolos_probabilidades = {simbolo: freq / total_simbolos for simbolo, freq in tabla_frecuencias.items()}
    
    # Crear la fuente discreta sin memoria
    fuente = FuenteDiscretaSinMemoria(simbolos_probabilidades)
    
    # Generar una cadena de símbolos de 1000 caracteres
    cadena_generada = fuente.generar_cadena(1000)
    
    # Crear el codificador Huffman basado en las frecuencias
    huffman = Huffman(tabla_frecuencias)
    
    # Codificar la cadena generada
    cadena_codificada = huffman.codifica(cadena_generada)
    
    # Evaluar el rendimiento del codificador
    evaluar_rendimiento(cadena_generada, cadena_codificada, simbolos_probabilidades, huffman)
  
    # Guardar los códigos binarios generados
    guardar_codigos_binarios(huffman.codigo)
    
    # Guardar el árbol de Huffman generado
    guardar_arbol_huffman(huffman.arbol, codigo=huffman.codigo)

In [43]:
ejecutar_objetivo_2_4()

Entropía de la fuente: 4.4678 bits/símbolo
Tamaño original: 8000 bits
Tamaño codificado: 4422 bits
Tasa de compresión: 1.8091
Longitud media del código de Huffman: 4.5083 bits/símbolo
Cumple con el Teorema de Codificación de Fuente.
Códigos binarios guardados en 'codigos_binarios.txt'.
